# Import Dependencies

In [2]:
# Import gym stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# Import helpers
import numpy as np
import random
import os

# Import Stable baselines stuff 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


# Type of Spaces

In [28]:
Discrete(3).sample()

1

In [34]:
Box(0, 1, shape=(3, 3)).sample()

array([[0.89639187, 0.5929647 , 0.03076255],
       [0.11553282, 0.22215818, 0.42589325],
       [0.23927933, 0.09014426, 0.47092372]], dtype=float32)

In [48]:
Tuple((Discrete(5, start=2), Box(-5, 5, shape=(5, ), dtype=np.int8))).sample()

(5, array([ 1,  4, -2, -1, -4], dtype=int8))

In [64]:
Dict({'height': Discrete(3), 'directions': Box(np.array([-1, 0, 0]), 1)}).sample()

OrderedDict([('directions',
              array([0.5876215 , 0.09161273, 0.05953738], dtype=float32)),
             ('height', 2)])

In [65]:
MultiBinary(4).sample()

array([0, 1, 0, 1], dtype=int8)

In [82]:
MultiDiscrete([5, 2, 2, 6]).sample()

array([1, 0, 0, 4], dtype=int64)

# Build an Environment
- Build and agent to give us the best shower possible
- Randomly temperaure
- 37 and 39 degree

In [3]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60

    def step(self, action):
        # Apply temp adj
        self.state += action - 1

        # Decrease shower time
        self.shower_length -= 1

        # Calculate Reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False

        info = {}
        
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [4]:
env = ShowerEnv()

c:\Users\david\OneDrive\Documents\dogzilla-robot-dog-2023\drl-sb3\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [11]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
check_env(env, warn=True)

# Test Environment

In [5]:
episodes = 5
for e in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample() 
        obs, reward, done, info = env.step(action)

        score += reward
    print(f'Episode: {e}, score: {score}')

Episode: 1, score: -28
Episode: 2, score: -16
Episode: 3, score: 24
Episode: 4, score: -52
Episode: 5, score: -40


# Train Model

In [9]:
log_path = os.path.join('training', 'logs')
log_path

'training\\logs'

In [13]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
model.learn(total_timesteps=40000)

Logging to training\logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -27.6    |
| time/              |          |
|    fps             | 1391     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -30         |
| time/                   |             |
|    fps                  | 968         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009939802 |
|    clip_fraction        | 0.0335      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000282   |

# Save Model

In [17]:
shower_path = os.path.join('training', 'saved models', 'project3_shower')
shower_path

'training\\saved models\\project3_shower'

In [18]:
model.save(shower_path)

In [128]:
del model

In [ ]:
model = PPO(shower_path, env)

# Evaluate the model

In [20]:
evaluate_policy(model, env, n_eval_episodes=10)

(-36.0, 48.0)